In [1]:
# ----------------- Import Libraries -----------------
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, classification_report

import seaborn as sns
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


2024-12-04 14:30:04.395655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-04 14:30:05.561098: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# ----------------- CBAM Block Definition -----------------
def cbam_block(inputs, reduction_ratio=0.5):
    channels = inputs.shape[-1]

    # Channel Attention
    avg_pool = layers.GlobalAveragePooling2D()(inputs)
    max_pool = layers.GlobalMaxPooling2D()(inputs)
    shared_layer_1 = layers.Dense(int(channels * reduction_ratio), activation='relu', use_bias=True)
    shared_layer_2 = layers.Dense(channels, activation='relu', use_bias=True)

    avg_pool = shared_layer_1(avg_pool)
    avg_pool = shared_layer_2(avg_pool)
    max_pool = shared_layer_1(max_pool)
    max_pool = shared_layer_2(max_pool)

    attention = layers.Add()([avg_pool, max_pool])
    attention = layers.Activation('sigmoid')(attention)
    attention = layers.Reshape((1, 1, channels))(attention)
    scaled_inputs = layers.Multiply()([inputs, attention])

    # Spatial Attention
    squeeze = layers.Conv2D(filters=1, kernel_size=1, activation='sigmoid', use_bias=False)(scaled_inputs)
    expanded_inputs = layers.Multiply()([scaled_inputs, squeeze])

    return expanded_inputs

In [3]:
def VGG19_CBAM_Model():
    input_shape = (224, 224, 3)
    vgg19 = tf.keras.applications.VGG19(include_top=False, input_shape=input_shape, weights='imagenet')

    x = cbam_block(vgg19.output)  # Apply CBAM block
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(units=1024, activation='relu')(x)
    x = layers.Dense(units=512, activation='relu')(x)
    x = layers.Dense(units=256, activation='relu')(x)
    x = layers.Dense(units=128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(units=4, activation='softmax')(x)

    model = models.Model(inputs=vgg19.input, outputs=x, name='VGG19_CBAM_Model')
    return model


In [4]:
# ----------------- Compile and Train -----------------
def compile_and_train(model, train_data, test_data, epochs=10, learning_rate=0.001):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = model.fit(
        train_data,
        validation_data=test_data,
        epochs=epochs,
        callbacks=[early_stopping],
        verbose=1
    )
    return history

In [5]:
def evaluate_global_model(model, test_loader, client_idx, output_dir="VGG19_model_results"):
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists
    output_file = os.path.join(output_dir, f"Model_Results_Client_{client_idx + 1}.txt")
    
    all_preds = []
    all_labels = []
    total_loss = 0

    for features, labels in test_loader:
        features_tf = tf.convert_to_tensor(features, dtype=tf.float32)

        # Convert one-hot encoded labels to class indices
        if len(labels.shape) > 1 and labels.shape[-1] > 1:  # Likely one-hot
            labels_tf = tf.argmax(labels, axis=1, output_type=tf.int32)
        else:
            labels_tf = tf.convert_to_tensor(labels, dtype=tf.int32)

        # Debugging: Print shapes
        print(f"Features shape: {features_tf.shape}, Labels shape: {labels_tf.shape}")

        # Get predictions
        predictions = model(features_tf, training=False)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)

        # Accumulate predictions and labels
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels_tf.numpy())


        # Compute loss
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_tf, logits=predictions)
        total_loss += tf.reduce_mean(loss).numpy()

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    # Classification report
    class_names = ["glioma", "meningioma", "notumor", "pituitary"] 
    classification_report_str = classification_report(all_labels, all_preds, target_names=class_names)

    # Append metrics to a file
    with open(output_file, "w") as file:
        file.write(f"Evaluation Metrics for Client {client_idx + 1}:\n")
        file.write(f"Accuracy: {accuracy * 100:.2f}%\n")
        file.write(f"Loss: {total_loss / len(test_loader):.4f}\n")
        file.write(f"Precision: {precision:.4f}\n")
        file.write(f"Recall: {recall:.4f}\n")
        file.write(f"F1 Score: {f1:.4f}\n\n")
        file.write("Classification Report:\n")
        file.write(classification_report_str)
        file.write("\nConfusion Matrix:\n")
        file.write("\n".join(["\t".join(map(str, row)) for row in conf_matrix]))
        file.write("\n" + "-" * 50 + "\n")

    # Print metrics
    print("\nClassification Report:\n", classification_report_str)
    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Loss: {total_loss / len(test_loader):.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Plot and save heatmap
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix (Client {client_idx + 1})')
    plt.savefig(os.path.join(output_dir, f"confusion_matrix_Client_{client_idx + 1}.png"))
    plt.close()


In [6]:
# Directories for each client
split_dirs = [
    "../NON_IID/Model_1",
    "../NON_IID/Model_2",
    "../NON_IID/Model_3",
    "../NON_IID/Model_4"
]

In [7]:
# Updated image transformations: Normalize first, then permute
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

def get_tumor_dataloaders(split_dirs, batch_size=25, shuffle=True):
    """
    Returns data loaders for all clients for both training and testing sets.
    """
    tumor_iid_train_dls = []
    tumor_iid_test_dls = []

    for client_idx, client_dir in enumerate(split_dirs):
        # Get the directory for the current client
        train_dir = os.path.join(client_dir, 'train')
        test_dir = os.path.join(client_dir, 'test')

        # Check if the directories exist
        if not os.path.exists(train_dir) or not os.path.exists(test_dir):
            print(f"Directory not found for client {client_idx + 1}:")
            print(f"Train dir: {train_dir}")
            print(f"Test dir: {test_dir}")
            continue

        # Load training data for the current client
        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

        # Load testing data for the current client
        test_dataset = datasets.ImageFolder(test_dir, transform=transform)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Append the dataloaders for the current client to the list
        tumor_iid_train_dls.append(train_loader)
        tumor_iid_test_dls.append(test_loader)

    return tumor_iid_train_dls, tumor_iid_test_dls

# Get the training and testing data loaders
Tumor_iid_train_dls, Tumor_iid_test_dls = get_tumor_dataloaders(split_dirs, batch_size=25)

# Checking the sizes of the images in the data loaders to verify the shape
for batch_idx, (images, labels) in enumerate(Tumor_iid_train_dls[0]):  # Checking for client 1
    print(f"Batch {batch_idx} image sizes: {images.size()}")  # Should print torch.Size([25, 224, 224, 3])
    break  # Check only the first batch


Batch 0 image sizes: torch.Size([25, 224, 224, 3])


In [8]:
from tensorflow.keras.utils import to_categorical

def pytorch_to_tf_dataset(pytorch_loader, num_classes):
    """
    Converts a PyTorch DataLoader to a TensorFlow dataset with one-hot encoded labels.
    """
    images, labels = [], []
    for batch_images, batch_labels in pytorch_loader:
        # Convert PyTorch tensors to NumPy arrays and permute to (Batch, Height, Width, Channels)
        images.append(batch_images.permute(0, 2, 1, 3).numpy())
        # One-hot encode labels
        labels.append(to_categorical(batch_labels.numpy(), num_classes=num_classes))
    
    # Concatenate all batches
    images = np.concatenate(images, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    # Create TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    return dataset.batch(pytorch_loader.batch_size)




In [9]:
if __name__ == "__main__":
    # Iterate over all client datasets
    for client_idx in range(len(split_dirs)):
        print(f"\nTraining and testing on dataset: Model_{client_idx + 1}\n")
        
        # Load data for the current client
        train_loader = Tumor_iid_train_dls[client_idx]
        test_loader = Tumor_iid_test_dls[client_idx]

        # Number of classes in your dataset
        num_classes = 4

        # Convert PyTorch DataLoaders to TensorFlow datasets
        tf_train_dataset = pytorch_to_tf_dataset(train_loader, num_classes)
        tf_test_dataset = pytorch_to_tf_dataset(test_loader, num_classes)

        # Build Model
        model = VGG19_CBAM_Model()

        # Use these datasets for training
        history = compile_and_train(model, tf_train_dataset, tf_test_dataset, epochs=10, learning_rate=0.0001)

        # Evaluate and Save Results
        evaluate_global_model(model, tf_test_dataset, client_idx, output_dir="VGG19_model_results")

        print(f"Training and evaluation completed for Model_{client_idx + 1}\n")



Training and testing on dataset: Model_1



2024-12-04 14:30:20.187103: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31132 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0


80134624/80134624 [==============================] - 6s 0us/step
Epoch 1/10


2024-12-04 14:30:33.349573: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1321,4]
	 [[{{node Placeholder/_1}}]]
2024-12-04 14:30:36.841857: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2024-12-04 14:30:41.929450: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f5324067170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-04 14:30:41.929490: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-12-04 14:30:41.935167: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12

53/53 [==============================] - ETA: 0s - loss: 0.6715 - accuracy: 0.7600

...

64/64 [==============================] - 53s 709ms/step - loss: 0.6079 - accuracy: 0.7605 - val_loss: 0.7963 - val_accuracy: 0.6948
Epoch 2/10
64/64 [==============================] - 35s 545ms/step - loss: 0.4005 - accuracy: 0.8188 - val_loss: 0.5599 - val_accuracy: 0.7221
Epoch 3/10
64/64 [==============================] - 35s 546ms/step - loss: 0.3629 - accuracy: 0.8458 - val_loss: 0.5103 - val_accuracy: 0.7384
Epoch 4/10
64/64 [==============================] - 35s 543ms/step - loss: 0.3185 - accuracy: 0.8652 - val_loss: 0.5494 - val_accuracy: 0.7657
Epoch 5/10
64/64 [==============================] - 35s 544ms/step - loss: 0.3047 - accuracy: 0.8621 - val_loss: 0.5886 - val_accuracy: 0.7657
Epoch 6/10
64/64 [==============================] - 35s 545ms/step - loss: 0.2974 - accuracy: 0.8702 - val_loss: 0.4467 - val_accuracy: 0.7847
Epoch 7/10
64/64 [==============================] - 35s 545ms/step - loss: 0.2746 - accuracy: 0.8803 - val_loss: 0.4250 - val_accuracy: 0.7684
Epoch 8/10

2024-12-04 14:48:50.576753: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1457,4]
	 [[{{node Placeholder/_1}}]]


59/59 [==============================] - ETA: 0s - loss: 0.6386 - accuracy: 0.7865

2024-12-04 14:49:32.132338: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [262,4]
	 [[{{node Placeholder/_1}}]]


59/59 [==============================] - 46s 657ms/step - loss: 0.6386 - accuracy: 0.7865 - val_loss: 0.7076 - val_accuracy: 0.6756
Epoch 2/10
59/59 [==============================] - 31s 532ms/step - loss: 0.3989 - accuracy: 0.8483 - val_loss: 0.7389 - val_accuracy: 0.7863
Epoch 3/10
59/59 [==============================] - 31s 532ms/step - loss: 0.3638 - accuracy: 0.8703 - val_loss: 0.7830 - val_accuracy: 0.7786
Epoch 4/10
59/59 [==============================] - 31s 533ms/step - loss: 0.3191 - accuracy: 0.8881 - val_loss: 0.6753 - val_accuracy: 0.8206
Epoch 5/10
59/59 [==============================] - 31s 532ms/step - loss: 0.2521 - accuracy: 0.9218 - val_loss: 0.6092 - val_accuracy: 0.8168
Epoch 6/10
59/59 [==============================] - 31s 532ms/step - loss: 0.2418 - accuracy: 0.9218 - val_loss: 0.4005 - val_accuracy: 0.8435
Epoch 7/10
59/59 [==============================] - 31s 532ms/step - loss: 0.2251 - accuracy: 0.9341 - val_loss: 0.3847 - val_accuracy: 0.8740
Epoch 8/10